## Evaluation of Classification Model


### Problem of Classification without Evaluation

In [1]:
import pandas as pd


# read titanic data
data = pd.read_csv("train.csv")
print(data.columns)

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


In [2]:
import numpy as np
from sklearn.base import BaseEstimator
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score


# create custom estimator
class DummyClassifier(BaseEstimator):
    def fit(self, X, y=None):
        pass

    def predict(self, X):
        # create default list with row len(X.shape[0]) and column 1
        pred = np.zeros( (X.shape[0], 1), dtype="int32" )
        for i in range(X.shape[0]):
            pred[i] = 0 if X["Sex"].iloc[i] == "male" else 1
        
        return np.array(pred)
        
# define feature and target
Y_data = data[["Survived"]]
X_data = data.drop(["Survived"], axis=1)

# split train and test split
X_train, X_test, Y_train, Y_test = train_test_split(X_data, Y_data, test_size=0.2, random_state=0)

model = DummyClassifier()
model.fit(X_train, Y_train)
pred = model.predict(X_test)
score = accuracy_score(pred, Y_test)

# accuracy: apprx 78%
print(score)

0.7877094972067039


- Simple Estimator can predict test set as 78%, so using accuracy_score is very dangerous.



In [3]:
# problem demonstration with imbalanced dataset
from sklearn.datasets import load_digits

# read MNIST dataset
# digits: <sklearn.utils._bunch.Bunch>
digits = load_digits()

In [4]:
# create custom estimator
class DummyEstimator2(BaseEstimator):
    def fit(self, X, Y=None):
        pass

    def predict(self, X):
        return np.zeros( (len(X), 1), dtype=bool)
        
# y: create imbalance targets
y = (digits.target==7).astype(int)
X_train, X_test, Y_train, Y_test = train_test_split(digits.data, y, random_state=11)
print(f"Size of Test Set: {X_test.shape}")
print(f"Distribution of Test Set\n{pd.Series(Y_test).value_counts()}")

Size of Test Set: (450, 64)
Distribution of Test Set
0    405
1     45
Name: count, dtype: int64


In [5]:
model = DummyEstimator2()
model.fit(X_train, Y_train)
pred = model.predict(X_test)
print(f"Accuracy Score: {np.round(accuracy_score(pred, Y_test))}")

Accuracy Score: 1.0


* Only accuracy metric can not be used to evaluate the result of prediction.

### Confusion Matrix

<table>
    <tr>
        <td></td>
        <td>Predicted as Negative(0)</td>
        <td>Predicted as Positive(1)</td>
    </tr>
    <tr>
        <td>Real as Negative(0)</td>
        <td>True Negative:
Right Prediction to Negative</td>
        <td>False Positive:
False Prediction to Negative</td>
    </tr>
    <tr>
        <td>Real as Positive(1)</td>
        <td>False Negative:
False Prediction to Positive</td>
        <td>True Positive:
True Prediction to Positive</td>
    </tr>
</table>

* accuracy  = (TP + FN) / (TP + FP + FN + TN)
* precision = TP / (FP + TP)
* recall = TP / (FN + TP)